# **💁🏻🗨️💁🏻‍♂️대화 요약 Baseline code**
> **Dialogue Summarization** 경진대회에 오신 여러분 환영합니다! 🎉    
> 본 대회에서는 최소 2명에서 최대 7명이 등장하여 나누는 대화를 요약하는 BART 기반 모델의 baseline code를 제공합니다.     
> 주어진 데이터를 활용하여 일상 대화에 대한 요약을 효과적으로 생성하는 모델을 만들어봅시다!

## ⚙️ 데이터 및 환경설정

### 1) 필요한 라이브러리 설치

- 필요한 라이브러리를 설치한 후 불러옵니다.

In [1]:
import pandas as pd
import os
import re
import json
import yaml
from glob import glob
from tqdm import tqdm
from pprint import pprint
import torch
import pytorch_lightning as pl
from rouge import Rouge # 모델의 성능을 평가하기 위한 라이브러리입니다.

from torch.utils.data import Dataset , DataLoader
from transformers import AutoTokenizer, BartForConditionalGeneration, BartConfig
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

import wandb # 모델 학습 과정을 손쉽게 Tracking하고, 시각화할 수 있는 라이브러리입니다.

In [ ]:
# ===========================================================
# 🔼 여기: import 구문 바로 아래에 삽입
# ===========================================================

import re

def clean_generated_summary(summary_list, remove_tokens):
    escaped_tokens = [re.escape(token) for token in remove_tokens]
    pattern = "|".join(escaped_tokens)

    cleaned_texts = []
    for text in summary_list:
        cleaned = re.sub(pattern, " ", text)
        cleaned = re.sub(r"\s+", " ", cleaned)
        cleaned = cleaned.strip()
        cleaned_texts.append(cleaned)

    return cleaned_texts

In [ ]:
# ============================================================
# 🔥 N-gram 반복 제거 함수 (요약문 품질 개선)
# ============================================================

import re

def remove_repeated_ngrams(text, n=3):
    """
    문장 내에서 반복되는 N-gram을 제거하는 함수.
    예: "오늘 날씨가 좋아요 오늘 날씨가 좋아요" → "오늘 날씨가 좋아요"
    """
    tokens = text.split()
    seen = set()
    result = []
    
    for i in range(len(tokens)):
        # 현재 N-gram 추출
        ngram = tuple(tokens[i:i+n])
        
        if len(ngram) < n:
            # 마지막 부분 단어는 그대로 추가
            result.append(tokens[i])
            continue
        
        # N-gram 반복 여부 확인
        if ngram not in seen:
            seen.add(ngram)
            result.append(tokens[i])
        else:
            # 중복된 N-gram이면 skip
            continue

    # 정리된 문장 반환
    return " ".join(result)


def clean_repetition_in_summaries(summary_list, n=3):
    """
    여러 요약문에 대해 N-gram 반복을 제거하는 wrapper 함수.
    """
    cleaned_list = []
    for text in summary_list:
        cleaned = remove_repeated_ngrams(text, n=n)
        
        # 다중 공백 정리
        cleaned = re.sub(r"\s+", " ", cleaned).strip()
        cleaned_list.append(cleaned)

    return cleaned_list


In [2]:
wandb.login(key="624410f236117b79b2ee07a4ccef31c7514a5e03")
print("✅ wandb 로그인 완료")


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: quriquri7 (fc_bootcamp). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /data/ephemeral/home/.netrc


✅ wandb 로그인 완료


In [3]:
print("✅ pandas version:", pd.__version__)
print("✅ torch version:", torch.__version__)
print("✅ transformers version:", __import__("transformers").__version__)
print("✅ CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))

✅ pandas version: 2.1.4
✅ torch version: 2.1.2+cu121
✅ transformers version: 4.35.2
✅ CUDA available: True
GPU name: NVIDIA GeForce RTX 3090


In [4]:
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

In [5]:
try:
    import wandb
    WANDB_AVAILABLE = True
except ImportError:
    WANDB_AVAILABLE = False
    print("⚠️ wandb not installed. Will run without experiment tracking.")

### 2) Config file 만들기 (선택)
- 모델 생성에 필요한 다양한 매개변수 정보를 저장할 수 있습니다.  
  따라서, 코드 상에서 모델의 매개변수를 설정할 수도 있지만 독립적인 매개변수 정보 파일을 생성하여 관리할 수 있습니다.

In [6]:
# config 설정에 tokenizer 모듈이 사용되므로 미리 tokenizer를 정의해줍니다.
tokenizer = AutoTokenizer.from_pretrained("digit82/kobart-summarization")

/data/ephemeral/home/nlp_server/venv310/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
config_data = {
    "general": {
        "data_path": "./data/", # 모델 생성에 필요한 데이터 경로를 사용자 환경에 맞게 지정합니다.
        "model_name": "digit82/kobart-summarization", # 불러올 모델의 이름을 사용자 환경에 맞게 지정할 수 있습니다.
        "output_dir": "./prediction/" # 모델의 최종 출력 값을 저장할 경로를 설정합니다.
    },
    "tokenizer": {
        "encoder_max_len": 512,
        "decoder_max_len": 100,
        "bos_token": f"{tokenizer.bos_token}",
        "eos_token": f"{tokenizer.eos_token}",
        # 특정 단어들이 분해되어 tokenization이 수행되지 않도록 special_tokens을 지정해줍니다.
        "special_tokens": ['#Person1#', '#Person2#', '#Person3#', '#PhoneNumber#', '#Address#', '#PassportNumber#']
    },
    "training": {
        "overwrite_output_dir": True,
        "num_train_epochs": 15,
        "learning_rate": 1e-5,
        "per_device_train_batch_size": 8,
        "per_device_eval_batch_size": 16,
        "warmup_ratio": 0.1,
        "weight_decay": 0.01,
        "lr_scheduler_type": 'cosine',
        "optim": 'adamw_torch',
        "gradient_accumulation_steps": 1,
        "evaluation_strategy": 'epoch',
        "save_strategy": 'epoch',
        "save_total_limit": 5,
        "fp16": True,
        "load_best_model_at_end": True,
        "seed": 42,
        "logging_dir": "./logs",
        "logging_strategy": "epoch",
        "predict_with_generate": True,
        "generation_max_length": 100,
        "do_train": True,
        "do_eval": True,
        "early_stopping_patience": 3,
        "early_stopping_threshold": 0.001,
        "report_to": "wandb" # (선택) wandb를 사용할 때 설정합니다.
    },
    # (선택) wandb 홈페이지에 가입하여 얻은 정보를 기반으로 작성합니다.
    "wandb": {
        "entity": "fc_bootcamp",
        "project": "huggingface",
        "name": "<run_id>"
    },
    "inference": {
        "ckt_path": "model ckt path", # 사전 학습이 진행된 모델의 checkpoint를 저장할 경로를 설정합니다.
        "result_path": "./prediction/",
        "no_repeat_ngram_size": 2,
        "early_stopping": True,
        "generate_max_length": 100,
        "num_beams": 4,
        "batch_size" : 32,
        # 정확한 모델 평가를 위해 제거할 불필요한 생성 토큰들을 정의합니다.
        "remove_tokens": ['<usr>', f"{tokenizer.bos_token}", f"{tokenizer.eos_token}", f"{tokenizer.pad_token}"]
    }
}

- 참고✅    
: wandb 라이브러리를 사용하기 위해선 entity, project, name를 지정해주어야 합니다. wandb 홈페이지에 가입한 후 얻은 정보를 입력하여 작동할 수 있습니다.

In [8]:
# 모델의 구성 정보를 YAML 파일로 저장합니다.
config_path = "config/config.yaml"
with open(config_path, "w") as file:
    yaml.dump(config_data, file, allow_unicode=True)

In [9]:
import os

os.makedirs("config", exist_ok=True)
config_path = "config/config.yaml"
with open(config_path, "w") as file:
    yaml.dump(config_data, file, allow_unicode=True)

print("✅ Config saved at:", config_path)


✅ Config saved at: config/config.yaml


In [10]:
with open(config_path, "r") as f:
    loaded_config = yaml.safe_load(f)

pprint(loaded_config)
print("✅ model_name:", loaded_config["general"]["model_name"])
print("✅ data_path :", loaded_config["general"]["data_path"])
print("✅ bos_token:", loaded_config["tokenizer"]["bos_token"])
print("✅ special_tokens:", loaded_config["tokenizer"]["special_tokens"])


{'general': {'data_path': './data/',
             'model_name': 'digit82/kobart-summarization',
             'output_dir': './prediction/'},
 'inference': {'batch_size': 32,
               'ckt_path': 'model ckt path',
               'early_stopping': True,
               'generate_max_length': 100,
               'no_repeat_ngram_size': 2,
               'num_beams': 4,
               'remove_tokens': ['<usr>', '<s>', '</s>', '<pad>'],
               'result_path': './prediction/'},
 'tokenizer': {'bos_token': '<s>',
               'decoder_max_len': 100,
               'encoder_max_len': 512,
               'eos_token': '</s>',
               'special_tokens': ['#Person1#',
                                  '#Person2#',
                                  '#Person3#',
                                  '#PhoneNumber#',
                                  '#Address#',
                                  '#PassportNumber#']},
 'training': {'do_eval': True,
              'do_train': True,
    

In [11]:
print("원래 vocab size:", tokenizer.vocab_size)
print("추가 special tokens:", config_data["tokenizer"]["special_tokens"])


원래 vocab size: 30000
추가 special tokens: ['#Person1#', '#Person2#', '#Person3#', '#PhoneNumber#', '#Address#', '#PassportNumber#']


### 3) Configuration 불러오기

In [12]:
# 저장된 config 파일을 불러옵니다.
config_path = "config/config.yaml"

with open(config_path, "r") as file:
    loaded_config = yaml.safe_load(file)

# 불러온 config 파일의 전체 내용을 확인합니다.
pprint(loaded_config)

{'general': {'data_path': './data/',
             'model_name': 'digit82/kobart-summarization',
             'output_dir': './prediction/'},
 'inference': {'batch_size': 32,
               'ckt_path': 'model ckt path',
               'early_stopping': True,
               'generate_max_length': 100,
               'no_repeat_ngram_size': 2,
               'num_beams': 4,
               'remove_tokens': ['<usr>', '<s>', '</s>', '<pad>'],
               'result_path': './prediction/'},
 'tokenizer': {'bos_token': '<s>',
               'decoder_max_len': 100,
               'encoder_max_len': 512,
               'eos_token': '</s>',
               'special_tokens': ['#Person1#',
                                  '#Person2#',
                                  '#Person3#',
                                  '#PhoneNumber#',
                                  '#Address#',
                                  '#PassportNumber#']},
 'training': {'do_eval': True,
              'do_train': True,
    

In [13]:
# 실험에 쓰일 데이터의 경로, 사용될 모델, 모델의 최종 출력 결과를 저장할 경로에 대해 확인합니다.
loaded_config['general']

{'data_path': './data/',
 'model_name': 'digit82/kobart-summarization',
 'output_dir': './prediction/'}

In [14]:
# 이곳에 사용자가 저장한 데이터 dir 설정하기
# loaded_config['general']['data_path'] = "data_path"

# evaluation_strategy -> eval_strategy 변환 (transformers 버전 호환성)
#if 'evaluation_strategy' in loaded_config['training']:
#     loaded_config['training']['eval_strategy'] = loaded_config['training'].pop('evaluation_strategy')

In [15]:
# 데이터 전처리를 하기 위해 tokenization 과정에서 필요한 정보들을 확인합니다.
loaded_config['tokenizer']

{'bos_token': '<s>',
 'decoder_max_len': 100,
 'encoder_max_len': 512,
 'eos_token': '</s>',
 'special_tokens': ['#Person1#',
  '#Person2#',
  '#Person3#',
  '#PhoneNumber#',
  '#Address#',
  '#PassportNumber#']}

In [16]:
# 모델이 훈련 시 적용될 매개변수를 확인합니다.
loaded_config['training']

{'do_eval': True,
 'do_train': True,
 'early_stopping_patience': 3,
 'early_stopping_threshold': 0.001,
 'evaluation_strategy': 'epoch',
 'fp16': True,
 'generation_max_length': 100,
 'gradient_accumulation_steps': 1,
 'learning_rate': 1e-05,
 'load_best_model_at_end': True,
 'logging_dir': './logs',
 'logging_strategy': 'epoch',
 'lr_scheduler_type': 'cosine',
 'num_train_epochs': 20,
 'optim': 'adamw_torch',
 'overwrite_output_dir': True,
 'per_device_eval_batch_size': 16,
 'per_device_train_batch_size': 8,
 'predict_with_generate': True,
 'report_to': 'wandb',
 'save_strategy': 'epoch',
 'save_total_limit': 5,
 'seed': 42,
 'warmup_ratio': 0.1,
 'weight_decay': 0.01}

In [17]:
# 모델 학습 과정에 대한 정보를 제공해주는 wandb 설정 내용을 확인합니다.
loaded_config['wandb']

{'entity': 'quriquri7', 'name': 'fc_bootcamp', 'project': 'nlp'}

In [ ]:
# (선택) 이곳에 사용자가 사용할 wandb config 설정
loaded_config['wandb']['entity'] = "fc_bootcamp"
loaded_config['wandb']['project'] = "huggingface"
loaded_config['wandb']['name'] = "<run_id>"

# wandb 비활성화 (권한 문제 해결)
loaded_config['training']['report_to'] = "wandb"

In [19]:
# 모델이 최종 결과를 출력하기 위한 매개변수 정보를 확인합니다.
loaded_config['inference']

{'batch_size': 32,
 'ckt_path': 'model ckt path',
 'early_stopping': True,
 'generate_max_length': 100,
 'no_repeat_ngram_size': 2,
 'num_beams': 4,
 'remove_tokens': ['<usr>', '<s>', '</s>', '<pad>'],
 'result_path': './prediction/'}

In [20]:
USE_WANDB = loaded_config["training"]["report_to"] == "wandb"

if USE_WANDB:
    wandb.init(
        entity=loaded_config['wandb']['entity'],
        project=loaded_config['wandb']['project'],
        name=loaded_config['wandb']['name'],
    )
    os.environ["WANDB_LOG_MODEL"] = "true"
    os.environ["WANDB_WATCH"] = "false"
else:
    os.environ["WANDB_DISABLED"] = "true"
    print("🚫 wandb 비활성화 상태로 학습을 진행합니다.")


🚫 wandb 비활성화 상태로 학습을 진행합니다.


In [21]:
import os

data_path = loaded_config["general"]["data_path"]
print("📂 data_path:", data_path)

if not os.path.exists(data_path):
    print("❌ data_path 경로가 없습니다. 경로를 다시 확인하세요.")
else:
    print("✅ data_path 존재")
    print("📄 파일 목록:", os.listdir(data_path))


📂 data_path: ./data/
✅ data_path 존재
📄 파일 목록: ['sample_submission.csv', 'train.csv', 'test.csv', 'dev.csv']


In [22]:
from transformers import AutoTokenizer, BartForConditionalGeneration

model_name = loaded_config["general"]["model_name"]
print("🔍 model_name:", model_name)

tok = AutoTokenizer.from_pretrained(model_name)
mdl = BartForConditionalGeneration.from_pretrained(model_name)
print("✅ model/tokenizer 로드 성공")


🔍 model_name: digit82/kobart-summarization


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


✅ model/tokenizer 로드 성공


### 4) 데이터 불러와서 확인해보기
- 실험에서 쓰일 데이터를 load하여 데이터의 구조와 내용을 살펴보겠습니다.
- Train, dev, test 순서대로 12457, 499, 250개 씩 데이터가 구성되어 있습니다.

In [23]:
# config에 저장된 데이터 경로를 통해 train과 validation data를 불러옵니다.
data_path = loaded_config['general']['data_path']

# train data의 구조와 내용을 확인합니다.
train_df = pd.read_csv(os.path.join(data_path,'train.csv'))
train_df.tail()

,fname,dialogue,summary,topic
12452,train_12455,#Person1#: 안녕하세요. 혹시 맨체스터에서 오신 Mr. Green 맞으신가요...,Tan Ling은 흰머리와 수염이 특징인 Mr. Green을 맞이하여 호텔로 안내합...,호텔 안내
12453,train_12456,"#Person1#: Mister Ewing이 우리 회의장에 4시에 오라고 했지, 맞...",#Person1#과 #Person2#는 Mister Ewing의 요청에 따라 회의장...,회의 준비
12454,train_12457,#Person1#: 오늘 어떻게 도와드릴까요?\n#Person2#: 차를 빌리고 싶...,#Person2#는 #Person1#의 도움으로 5일 동안 소형차를 대여합니다.,차량 대여
12455,train_12458,#Person1#: 너 오늘 좀 기분 안 좋아 보인다? 무슨 일 있어?\n#Pers...,#Person2#의 어머니가 직장을 잃으셨다. #Person2#는 어머니가 우울해하...,실직과 대처
12456,train_12459,"#Person1#: 엄마, 나 다음 주 토요일에 이모부네 가족 보러 가는데, 오늘 ...",#Person1#은 다음 주 토요일에 이모부네 가족을 방문하기 위해 짐을 싸야 하는...,가족 방문 준비


In [24]:
# validation data의 구조와 내용을 확인합니다.
val_df = pd.read_csv(os.path.join(data_path,'dev.csv'))
val_df.tail()

,fname,dialogue,summary,topic
494,dev_495,#Person1#: 새해가 되니까 나도 새 출발을 하기로 했어.\n#Person2#...,#Person1#은 새해에 담배를 끊고 커밍아웃 하기로 결심했습니다. #Person...,새해 결심
495,dev_496,#Person1#: 너 Joe랑 결혼했지?\n#Person2#: Joe? 무슨 말이...,"#Person1#은 #Person2#가 Joe와 결혼했다고 생각하지만, #Perso...",사랑과 결혼 오해
496,dev_497,"#Person1#: 어떻게 도와드릴까요, 아줌마?\n#Person2#: 제 차에서 ...","#Person2#의 차에서 소리가 나며, 브레이크 수리가 필요한 상황입니다. #Pe...",차량 소음 및 수리
497,dev_498,"#Person1#: 여보세요, 아마존 고객 서비스입니다. 어떻게 도와드릴까요?\n#...",#Person2#가 아마존 고객 서비스에 전화하여 아마존에서 구매한 책에 53페이지...,책 페이지 누락
498,dev_499,#Person1#: 벌써 여름이 다가오다니 믿기지 않아. \n#Person2#: 맞...,"#Person2#는 여름방학 동안 파티에서 일하는 회사에서 일하며, 주로 음식 준비...",여름방학 일자리


## 1. 데이터 가공 및 데이터셋 클래스 구축
- csv file 을 불러와서 encoder 와 decoder의 입력형태로 가공해줍니다.
- 가공된 데이터를 torch dataset class 로 구축하여 모델에 입력가능한 형태로 만듭니다.

In [25]:
# 데이터 전처리를 위한 클래스로, 데이터셋을 데이터프레임으로 변환하고 인코더와 디코더의 입력을 생성합니다.
class Preprocess:
    def __init__(self,
            bos_token: str,
            eos_token: str,
        ) -> None:

        self.bos_token = bos_token
        self.eos_token = eos_token

    @staticmethod
    # 실험에 필요한 컬럼을 가져옵니다.
    def make_set_as_df(file_path, is_train = True):
        if is_train:
            df = pd.read_csv(file_path)
            train_df = df[['fname','dialogue','summary']]
            return train_df
        else:
            df = pd.read_csv(file_path)
            test_df = df[['fname','dialogue']]
            return test_df

    # BART 모델의 입력, 출력 형태를 맞추기 위해 전처리를 진행합니다.
    def make_input(self, dataset,is_test = False):
        if is_test:
            encoder_input = dataset['dialogue']
            decoder_input = [self.bos_token] * len(dataset['dialogue'])
            return encoder_input.tolist(), list(decoder_input)
        else:
            encoder_input = dataset['dialogue']
            decoder_input = dataset['summary'].apply(lambda x : self.bos_token + str(x)) # Ground truth를 디코더의 input으로 사용하여 학습합니다.
            decoder_output = dataset['summary'].apply(lambda x : str(x) + self.eos_token)
            return encoder_input.tolist(), decoder_input.tolist(), decoder_output.tolist()


In [26]:
# Train에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForTrain(Dataset):
    def __init__(self, encoder_input, decoder_input, labels, len):
        self.encoder_input = encoder_input
        self.decoder_input = decoder_input
        self.labels = labels
        self.len = len

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()} # item[input_ids], item[attention_mask]
        item2 = {key: val[idx].clone().detach() for key, val in self.decoder_input.items()} # item2[input_ids], item2[attention_mask]
        item2['decoder_input_ids'] = item2['input_ids']
        item2['decoder_attention_mask'] = item2['attention_mask']
        item2.pop('input_ids')
        item2.pop('attention_mask')
        item.update(item2) #item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask]
        item['labels'] = self.labels['input_ids'][idx] #item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask], item[labels]
        return item

    def __len__(self):
        return self.len

# Validation에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForVal(Dataset):
    def __init__(self, encoder_input, decoder_input, labels, len):
        self.encoder_input = encoder_input
        self.decoder_input = decoder_input
        self.labels = labels
        self.len = len

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()} # item[input_ids], item[attention_mask]
        item2 = {key: val[idx].clone().detach() for key, val in self.decoder_input.items()} # item2[input_ids], item2[attention_mask]
        item2['decoder_input_ids'] = item2['input_ids']
        item2['decoder_attention_mask'] = item2['attention_mask']
        item2.pop('input_ids')
        item2.pop('attention_mask')
        item.update(item2) #item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask]
        item['labels'] = self.labels['input_ids'][idx] #item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask], item[labels]
        return item

    def __len__(self):
        return self.len

# Test에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForInference(Dataset):
    def __init__(self, encoder_input, test_id, len):
        self.encoder_input = encoder_input
        self.test_id = test_id
        self.len = len

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()}
        item['ID'] = self.test_id[idx]
        return item

    def __len__(self):
        return self.len


In [27]:
# tokenization 과정까지 진행된 최종적으로 모델에 입력될 데이터를 출력합니다.
def prepare_train_dataset(config, preprocessor, data_path, tokenizer):
    train_file_path = os.path.join(data_path,'train.csv')
    val_file_path = os.path.join(data_path,'dev.csv')

    # train, validation에 대해 각각 데이터프레임을 구축합니다.
    train_data = preprocessor.make_set_as_df(train_file_path)
    val_data = preprocessor.make_set_as_df(val_file_path)

    print('-'*150)
    print(f'train_data:\n {train_data["dialogue"][0]}')
    print(f'train_label:\n {train_data["summary"][0]}')

    print('-'*150)
    print(f'val_data:\n {val_data["dialogue"][0]}')
    print(f'val_label:\n {val_data["summary"][0]}')

    encoder_input_train , decoder_input_train, decoder_output_train = preprocessor.make_input(train_data)
    encoder_input_val , decoder_input_val, decoder_output_val = preprocessor.make_input(val_data)
    print('-'*10, 'Load data complete', '-'*10,)

    tokenized_encoder_inputs = tokenizer(encoder_input_train, return_tensors="pt", padding=True,
                            add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False)
    tokenized_decoder_inputs = tokenizer(decoder_input_train, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)
    tokenized_decoder_ouputs = tokenizer(decoder_output_train, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)

    train_inputs_dataset = DatasetForTrain(tokenized_encoder_inputs, tokenized_decoder_inputs, tokenized_decoder_ouputs,len(encoder_input_train))

    val_tokenized_encoder_inputs = tokenizer(encoder_input_val, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False)
    val_tokenized_decoder_inputs = tokenizer(decoder_input_val, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)
    val_tokenized_decoder_ouputs = tokenizer(decoder_output_val, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)

    val_inputs_dataset = DatasetForVal(val_tokenized_encoder_inputs, val_tokenized_decoder_inputs, val_tokenized_decoder_ouputs,len(encoder_input_val))

    print('-'*10, 'Make dataset complete', '-'*10,)
    return train_inputs_dataset, val_inputs_dataset

## 2. Trainer 및 Trainingargs 구축하기
- Huggingface 의 Trainer 와 Training arguments를 활용하여 모델 학습을 일괄적으로 처리해주는 클래스를 정의합니다.

In [28]:
# 모델 성능에 대한 평가 지표를 정의합니다. 본 대회에서는 ROUGE 점수를 통해 모델의 성능을 평가합니다.
def compute_metrics(config,tokenizer,pred):
    rouge = Rouge()
    predictions = pred.predictions
    labels = pred.label_ids

    predictions[predictions == -100] = tokenizer.pad_token_id
    labels[labels == -100] = tokenizer.pad_token_id

    decoded_preds = tokenizer.batch_decode(predictions, clean_up_tokenization_spaces=True)
    labels = tokenizer.batch_decode(labels, clean_up_tokenization_spaces=True)

    # 정확한 평가를 위해 미리 정의된 불필요한 생성토큰들을 제거합니다.
    replaced_predictions = decoded_preds.copy()
    replaced_labels = labels.copy()
    remove_tokens = config['inference']['remove_tokens']
    for token in remove_tokens:
        replaced_predictions = [sentence.replace(token," ") for sentence in replaced_predictions]
        replaced_labels = [sentence.replace(token," ") for sentence in replaced_labels]

    print('-'*150)
    print(f"PRED: {replaced_predictions[0]}")
    print(f"GOLD: {replaced_labels[0]}")
    print('-'*150)
    print(f"PRED: {replaced_predictions[1]}")
    print(f"GOLD: {replaced_labels[1]}")
    print('-'*150)
    print(f"PRED: {replaced_predictions[2]}")
    print(f"GOLD: {replaced_labels[2]}")

    # 최종적인 ROUGE 점수를 계산합니다.
    results = rouge.get_scores(replaced_predictions, replaced_labels,avg=True)

    # ROUGE 점수 중 F-1 score를 통해 평가합니다.
    result = {key: value["f"] for key, value in results.items()}
    return result

In [ ]:
# 학습을 위한 trainer 클래스와 매개변수를 정의합니다.
def load_trainer_for_train(config,generate_model,tokenizer,train_inputs_dataset,val_inputs_dataset):
    print('-'*10, 'Make training arguments', '-'*10,)
    # 'evaluation_strategy'가 없으면 기본값 'epoch'을 사용
    evaluation_strategy = config['training'].get('evaluation_strategy', 'epoch')
    training_args = Seq2SeqTrainingArguments(
                output_dir=config['general']['output_dir'], # model output directory
                overwrite_output_dir=config['training']['overwrite_output_dir'],
                num_train_epochs=config['training']['num_train_epochs'],  # total number of training epochs
                learning_rate=config['training']['learning_rate'], # learning_rate
                per_device_train_batch_size=config['training']['per_device_train_batch_size'], # batch size per device during training
                per_device_eval_batch_size=config['training']['per_device_eval_batch_size'],# batch size for evaluation
                warmup_ratio=config['training']['warmup_ratio'],  # number of warmup steps for learning rate scheduler
                weight_decay=config['training']['weight_decay'],  # strength of weight decay
                lr_scheduler_type=config['training']['lr_scheduler_type'],
                optim =config['training']['optim'],
                gradient_accumulation_steps=config['training']['gradient_accumulation_steps'],
                evaluation_strategy=evaluation_strategy, # evaluation strategy to adopt during training
                save_strategy =config['training']['save_strategy'],
                save_total_limit=config['training']['save_total_limit'], # number of total save model.
                fp16=config['training']['fp16'],
                load_best_model_at_end=config['training']['load_best_model_at_end'], # 최종적으로 가장 높은 점수 저장
                seed=config['training']['seed'],
                logging_dir=config['training']['logging_dir'], # directory for storing logs
                logging_strategy=config['training']['logging_strategy'],
                predict_with_generate=config['training']['predict_with_generate'], #To use BLEU or ROUGE score
                generation_max_length=config['training']['generation_max_length'],
                do_train=config['training']['do_train'],
                do_eval=config['training']['do_eval'],
                report_to=config['training']['report_to'] # (선택) wandb를 사용할 때 설정합니다.
            )
    # (선택) wandb 사용 시에만 초기화
    if config['training']['report_to'] == 'wandb':
        wandb.init(
            entity=config['wandb']['entity'],
            project=config['wandb']['project'],
            name=config['wandb']['name'],
        )
        # 모델 checkpoint를 wandb에 저장하도록 환경 변수를 설정합니다.
        os.environ["WANDB_LOG_MODEL"]="true"
        os.environ["WANDB_WATCH"]="false"

    # Validation loss가 더 이상 개선되지 않을 때 학습을 중단시키는 EarlyStopping 기능을 사용합니다.
    MyCallback = EarlyStoppingCallback(
        early_stopping_patience=config['training']['early_stopping_patience'],
        early_stopping_threshold=config['training']['early_stopping_threshold']
    )
    print('-'*10, 'Make training arguments complete', '-'*10,)
    print('-'*10, 'Make trainer', '-'*10,)

    # Trainer 클래스를 정의합니다.
    trainer = Seq2SeqTrainer(
        model=generate_model, # 사용자가 사전 학습하기 위해 사용할 모델을 입력합니다.
        args=training_args,
        train_dataset=train_inputs_dataset,
        eval_dataset=val_inputs_dataset,
        compute_metrics = lambda pred: compute_metrics(config,tokenizer, pred),
        callbacks = [MyCallback]
    )
    print('-'*10, 'Make trainer complete', '-'*10,)

    return trainer

In [30]:
# 학습을 위한 tokenizer와 사전 학습된 모델을 불러옵니다.
def load_tokenizer_and_model_for_train(config,device):
    print('-'*10, 'Load tokenizer & model', '-'*10,)
    print('-'*10, f'Model Name : {config["general"]["model_name"]}', '-'*10,)
    model_name = config['general']['model_name']
    bart_config = BartConfig().from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    generate_model = BartForConditionalGeneration.from_pretrained(config['general']['model_name'],config=bart_config)

    special_tokens_dict={'additional_special_tokens':config['tokenizer']['special_tokens']}
    tokenizer.add_special_tokens(special_tokens_dict)

    generate_model.resize_token_embeddings(len(tokenizer)) # 사전에 special token을 추가했으므로 재구성 해줍니다.
    generate_model.to(device)
    print(generate_model.config)

    print('-'*10, 'Load tokenizer & model complete', '-'*10,)
    return generate_model , tokenizer

## 3. 모델 학습하기

- 앞에서 구축한 클래스 및 함수를 활용하여 학습 진행합니다.

In [ ]:
def main(config):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print('-'*10, f'device : {device}', '-'*10,)
    print(torch.__version__)

    # 1) 모델 + tokenizer 로드 (LoRA 적용된 버전)
    generate_model, tokenizer = load_tokenizer_and_model_for_train(config, device)

    # 2) 데이터셋 준비
    preprocessor = Preprocess(config['tokenizer']['bos_token'], config['tokenizer']['eos_token'])
    data_path = config['general']['data_path']
    train_inputs_dataset, val_inputs_dataset = prepare_train_dataset(
        config, preprocessor, data_path, tokenizer
    )

    # 3) Trainer 생성
    trainer = load_trainer_for_train(
        config, generate_model, tokenizer, train_inputs_dataset, val_inputs_dataset
    )

    # 4) 🔥 학습 "한 번" 실행 → 여기서 20 epoch 전체 돎
    trainer.train()

    # 5) 🔥 학습 끝난 LoRA 어댑터 저장
    save_dir = "./lora_adapter_final"
    generate_model.save_pretrained(save_dir)
    tokenizer.save_pretrained(save_dir)  # 토크나이저도 같이 저장해두면 편함

    # wandb 쓰면 종료
    if config['training']['report_to'] == 'wandb':
        wandb.finish()


In [32]:
if __name__ == "__main__":
    main(loaded_config)

---------- device : cuda:0 ----------
2.1.2+cu121
---------- Load tokenizer & model ----------
---------- Model Name : digit82/kobart-summarization ----------


/data/ephemeral/home/nlp_server/venv310/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


BartConfig {
  "_name_or_path": "digit82/kobart-summarization",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.1,
  "d_model": 768,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 1,
  "extra_pos_embeddings": 2,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 2,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "max_position_embeddings":

/data/ephemeral/home/nlp_server/venv310/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


---------- Make dataset complete ----------
---------- Make training arguments ----------
---------- Make training arguments complete ----------
---------- Make trainer ----------
---------- Make trainer complete ----------


Epoch,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l
1,2.923900,0.584783,0.328175,0.098915,0.305694
2,0.590100,0.537180,0.355463,0.121242,0.333007
3,0.523300,0.517694,0.369680,0.138448,0.349836
4,0.470000,0.517329,0.360902,0.130019,0.340327
5,0.427600,0.518148,0.364106,0.128896,0.339649
6,0.389500,0.519748,0.372131,0.137899,0.347269


------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:  #Person2# 는 감기에 걸렸습니다. #Person2# 는 천식 검사를 위해 폐 전문의에게 가보라고 권유합니다.                                        
GOLD: #Person2# 는 숨쉬기 어려워합니다. 의사는 #Person2# 에게 증상을 확인하고, 천식 검사를 위해 폐 전문의에게 가볼 것을 권합니다.                                                                     
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:  #Person1# 과 Jimmy에게 운동 계획을 묻고, Jimmy는 #Person1# 에게 #Person1# 의 운동 계획을 설명한다.                                    
GOLD: #Person1# 는 Jimmy를 운동하러 초대하고 팔과 복근 운동을 하도록 설득합니다.                                                                                
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:  #Person2# 는 건강에 좋

There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


## 4. 모델 추론하기

In [56]:
# 이곳에 내가 사용할 wandb config 설정
loaded_config['inference']['ckt_path'] = "./prediction/checkpoint-6232"

- test data를 사용하여 모델의 성능을 확인합니다.

In [58]:
# tokenization 과정까지 진행된 최종적으로 모델에 입력될 데이터를 출력합니다.
def prepare_test_dataset(config,preprocessor, tokenizer):

    test_file_path = os.path.join(config['general']['data_path'],'test.csv')

    test_data = preprocessor.make_set_as_df(test_file_path,is_train=False)
    test_id = test_data['fname']

    print('-'*150)
    print(f'test_data:\n{test_data["dialogue"][0]}')
    print('-'*150)

    encoder_input_test , decoder_input_test = preprocessor.make_input(test_data,is_test=True)
    print('-'*10, 'Load data complete', '-'*10,)

    test_tokenized_encoder_inputs = tokenizer(encoder_input_test, return_tensors="pt", padding=True,
                    add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False,)
    test_tokenized_decoder_inputs = tokenizer(decoder_input_test, return_tensors="pt", padding=True,
                    add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False,)

    test_encoder_inputs_dataset = DatasetForInference(test_tokenized_encoder_inputs, test_id, len(encoder_input_test))
    print('-'*10, 'Make dataset complete', '-'*10,)

    return test_data, test_encoder_inputs_dataset

In [ ]:
from peft import PeftModel
from transformers import AutoTokenizer, BartForConditionalGeneration

def load_tokenizer_and_model_for_test_lora(config, device):
    print('-'*10, 'Load tokenizer & LoRA model for inference', '-'*10)

    base_model_name = config['general']['model_name']  # "digit82/kobart-summarization"
    lora_path = config['inference']['ckt_path']        # 여기에 "./lora_adapter_final" 적어두기

    # 1) base tokenizer & special tokens
    tokenizer = AutoTokenizer.from_pretrained(base_model_name)
    special_tokens_dict = {'additional_special_tokens': config['tokenizer']['special_tokens']}
    tokenizer.add_special_tokens(special_tokens_dict)

    # 2) base KoBART 불러오기
    base_model = BartForConditionalGeneration.from_pretrained(base_model_name)
    base_model.resize_token_embeddings(len(tokenizer))

    # 3) LoRA adapter 입히기
    model = PeftModel.from_pretrained(base_model, lora_path)

    # 4) (선택) merge해서 추론 속도 빠르게
    model = model.merge_and_unload()

    model.to(device)
    print('-'*10, 'LoRA inference model load complete', '-'*10)
    return model, tokenizer

In [ ]:
# 학습된 모델이 생성한 요약문의 출력 결과를 보여줍니다.
def inference(config):
    device = torch.device('cuda:0' if torch.cuda.is_available()  else 'cpu')
    print('-'*10, f'device : {device}', '-'*10,)
    print(torch.__version__)

    generate_model , tokenizer = load_tokenizer_and_model_for_test_lora(config,device)

    data_path = config['general']['data_path']
    preprocessor = Preprocess(config['tokenizer']['bos_token'], config['tokenizer']['eos_token'])

    test_data, test_encoder_inputs_dataset = prepare_test_dataset(config,preprocessor, tokenizer)
    dataloader = DataLoader(test_encoder_inputs_dataset, batch_size=config['inference']['batch_size'])

    summary = []
    text_ids = []
    with torch.no_grad():
        for item in tqdm(dataloader):
            text_ids.extend(item['ID'])
            generated_ids = generate_model.generate(input_ids=item['input_ids'].to('cuda:0'),
                            no_repeat_ngram_size=config['inference']['no_repeat_ngram_size'],
                            early_stopping=config['inference']['early_stopping'],
                            max_length=config['inference']['generate_max_length'],
                            num_beams=config['inference']['num_beams'],
                        )
            for ids in generated_ids:
                result = tokenizer.decode(ids)
                summary.append(result)

    # 정확한 평가를 위하여 노이즈에 해당되는 스페셜 토큰을 제거합니다.
    remove_tokens = config['inference']['remove_tokens']
    preprocessed_summary = clean_generated_summary(summary, remove_tokens)

    # 🔥 추가: 반복 문장 제거
    preprocessed_summary = clean_repetition_in_summaries(preprocessed_summary, n=3)
  

    output = pd.DataFrame(
        {
            "fname": test_data['fname'],
            "summary" : preprocessed_summary,
        }
    )
    result_path = config['inference']['result_path']
    if not os.path.exists(result_path):
        os.makedirs(result_path)
    output.to_csv(os.path.join(result_path, "output.csv"), index=False)

    return output

In [64]:
# 학습된 모델의 test를 진행합니다.
if __name__ == "__main__":
    output = inference(loaded_config)

---------- device : cuda:0 ----------
2.1.2+cu121
---------- Load tokenizer & model ----------
---------- Model Name : digit82/kobart-summarization ----------


/data/ephemeral/home/nlp_server/venv310/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


---------- Load tokenizer & model complete ----------
------------------------------------------------------------------------------------------------------------------------------------------------------
test_data:
#Person1#: Ms. Dawson, 받아쓰기 좀 부탁드려야겠어요. 
#Person2#: 네, 말씀하세요... 
#Person1#: 이걸 오늘 오후까지 모든 직원들에게 사내 메모로 보내야 해요. 준비됐나요? 
#Person2#: 네, 말씀하세요. 
#Person1#: 모든 직원에게 알립니다... 즉시 발효되어 모든 사내 통신은 이메일과 공식 메모로만 제한됩니다. 근무 시간 동안 즉시 메시지 프로그램 사용은 금지됩니다. 
#Person2#: 이 정책이 사내 통신에만 적용되나요, 아니면 외부 통신에도 해당되나요? 
#Person1#: 이는 모든 통신에 적용됩니다. 사무실 내 직원 간 통신 뿐만 아니라 외부 통신도 해당됩니다. 
#Person2#: 하지만 많은 직원들이 고객과 소통하려고 즉시 메시지를 사용합니다. 
#Person1#: 통신 방법을 바꿔야 할 것입니다. 이 사무실에서는 즉시 메시지를 사용하는 것을 원하지 않습니다. 너무 많은 시간이 낭비됩니다! 이제 계속해서 메모를 작성해 주세요. 어디까지 했죠? 
#Person2#: 내외부 통신에 적용됩니다. 
#Person1#: 네. 즉시 메시지를 계속 사용하면 경고 후 시정 조치가 이루어지며, 두 번째 위반 시 해고될 수 있습니다. 이번 정책에 관한 질문은 부서장에게 문의하세요. 
#Person2#: 그게 다인가요? 
#Person1#: 네. 오늘 오후 4시까지 이 메모를 작성하고 배포해주세요.
----------------------------------------------------------------------------

100%|██████████| 16/16 [00:23<00:00,  1.47s/it]


In [55]:
output  # 각 대화문에 대한 요약문이 출력됨을 확인할 수 있습니다.

,fname,summary
0,test_0,Ms. Dawson은 #Person1# 에게 사내 메모를 작성하고 배포하라고 요청합니다.
1,test_1,#Person1# 은 #Person2# 에게 교통체증으로 인해 대중교통으로 출퇴근하...
2,test_2,Kate는 Masha와 Hero가 두 달 동안 별거 중이어서 이혼을 했다고 #Per...
3,test_3,Brian은 #Person1# 에게 생일 파티에서 춤을 추고 목걸이를 선물한다.
4,test_4,#Person1# 과 #Person2# 는 올림픽 공원의 크기에 대해 이야기하고 있다.
...,...,...
494,test_495,Jack과 Charlie는 새로운 비디오 게임을 하기로 결정했다.
495,test_496,#Person2# 는 #Person1# 에게 시골 음악에 관심을 갖게 된 계기와 시...
496,test_497,Alice는 세탁기 사용법을 #Person1# 에게 설명합니다. 세탁기는 비누를 따...
497,test_498,Steve는 Mrs. Thou에게 아파트를 보여줄 수 있는지 물어보고 다시 연락할 ...
